### Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
import nltk
from stop_words import get_stop_words
import string
from gensim import corpora, models
from stemming.porter2 import stem

### Text Preparation

In [2]:
# Importar dados limpos
ufrgs = pd.read_excel('dados_ufrgs_clean.xlsx')
ufrgs.index = range(ufrgs.shape[0])

# Amostra para teste
ufrgs = ufrgs[0:100]

# Definir o texto que será analisado: resumo ou título
analisar = 'abstract'

# Normalização: garantir que caracteres iguais (ou textos iguais) sejam representados pelo mesmo ponto de código
# Encode ASCII: errors='ignore' garante a remoção dos acentos, mantendo os caracteres puros
# Decode UTF-8: converte para UTF-8
# Punctuation: remover a pontuação
# Lower: converter os caracteres em minúsculo
# Stemming: não é um processo bem definido. Normalmente se utiliza o stemmer que melhor se adapta a sua ncessidade
# Stemming, ou Stemização, é a redução da palavra a sua raiz (stem)
ufrgs[analisar] = ufrgs[analisar].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.replace('[{}]'.format(string.punctuation), '').str.lower().str.split(' ').apply(lambda x: [stem(y) for y in x])

In [3]:
ufrgs.index = range(ufrgs.shape[0])

In [4]:
# Stopwords: são palavras que podem ser consideradas irrelevantes para o conjunto de palavras analisado
# As bibliotecas disponíveis em português não são completas. Portanto, são utilizadas mais de uma para enriquecer a lista
stopwords_1 = nltk.corpus.stopwords.words('portuguese')
stopwords_2 = get_stop_words('portuguese')

# Caso hajam termos em inglês no meio dos textos em portugês, stopwords em inglês são removidas
stopwords_3 = nltk.corpus.stopwords.words('english')

# Lista de palavras irrelevabtes que aparecem na etapa de teste/treinamento do modelo
stopwords_4 = ['pesquisa','análise','dados','sobre','através','partir','of','grande','in','deste','ser','forma','cada','meio','resultados',
               'além','identificar','quanto','presente','realizada','bem','avaliar','avaliação','principais','quais','base','sendo','fim',
               'amostra','coleta','experimento','material','materiais','método','métodos','metodologia','metodologias','objetivos','objetivo',
               'meta','problema','revisão','técnica','tópico','assim','criterio','feita','fator','porto','alegre','apresentada','apresentar',
               'ambos','capitulo','nesta','usuario','caracteristica','artigo','obtido','utilizado','parametro','grupo','maior','tres',
               'numero','atividade','rede','media','maior','variavel','significativa','analisar','utilizando','dentro','dentre','podem','analisado',
               'parte','obtido','ensaio','tipo','carga','possível','obtido','valor','aplicação','caso','serviço','plano','necessidade','setor','pode',
               'contra','conteúdo','tema','conceito','2018','outro','geral','elemento','aspecto','compreender','fisica']

# Junção das listas de stopwords
stopwords = stopwords_1 + stopwords_2 + stopwords_3 + stopwords_4

# Transformar a lista de stopwords em data frame para consumo
stopwords = pd.DataFrame(stopwords)

# Pré=tratamento das stopwords, sem necessidade de remover pontuação
stopwords[0] = stopwords[0].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().apply(lambda x: stem(x))

In [5]:
# Tokens são termos individuais (palavras), ou uma sequência de caracteres entre dois espaços, 
# ou entre um espaço e sinais de pontuação
lista_lista_token = list(ufrgs[analisar].values)

# Lista para guardar listas sem stopwords
lista_lista_token_ = []

# Loop para remover stopwords e tokens remanescentes com menos de quatro cracteres
for i in range(len(lista_lista_token)):
    lista_token = [token for token in lista_lista_token[i] if (token not in list(stopwords[0].values) and len(token) > 3)]
    lista_lista_token_.append(lista_token)

### LDA - Latent Dirichlet Allocation

In [19]:
# Gensim é uma biblioteca para topic modeling, document indexing and similarity retrieval de grande corpora
# Corpora (plural de corpus) onjunto de dados linguísticos coletados para serem objeto de análise

# Inicializar o dicionário de palavras (palavras:ids)
objeto_LDA = corpora.Dictionary(lista_lista_token_)

# Criar o corpus. Conversão do dicionário para o 'bag-of-words' (ids,contagem)
corpus = [objeto_LDA.doc2bow(lista_token) for lista_token in lista_lista_token_]

# Definir o número de tópicos
topicos = 10

# LDA é um modelo não-supervisionado de machine learning. O input é um texto (corpus) e o output são os tópicos
# O modelo possui três parâmetros principais: 
# 1. Número de tópicos;
# 2. Número de palavras por tópico eta = [x]*len(objeto_LDA.keys());
# 3. Número de tópicos por documento alpha = [y]*topicos.
lda_model = models.LdaModel(corpus, num_topics=topicos, id2word=objeto_LDA,passes=10, alpha='auto',eta='auto')

In [21]:
for topicos,palavras in lda_model.show_topics(formatted=True, num_topics=topicos):
    print("Tópico "+ str(topicos) + ": "+ palavras + "\n")

Tópico 0: 0.021*"empresa" + 0.021*"estrategico" + 0.018*"residuo" + 0.014*"torr" + 0.014*"area" + 0.014*"gestao" + 0.011*"colaborador" + 0.011*"geracao" + 0.011*"energia" + 0.011*"desempenho"

Tópico 1: 0.028*"social" + 0.026*"desenvolvimento" + 0.023*"gestao" + 0.018*"area" + 0.015*"cidad" + 0.013*"estabelecimento" + 0.013*"espaco" + 0.012*"consumidor" + 0.011*"locai" + 0.011*"alimentacao"

Tópico 4: 0.021*"pessoa" + 0.019*"empresa" + 0.016*"processo" + 0.013*"video" + 0.013*"joven" + 0.013*"politica" + 0.012*"contexto" + 0.012*"publico" + 0.011*"funcionario" + 0.011*"negocio"

Tópico 8: 0.043*"alimento" + 0.019*"control" + 0.018*"alimentacao" + 0.018*"seguranca" + 0.013*"especi" + 0.011*"area" + 0.011*"conservacao" + 0.011*"fibra" + 0.011*"densidad" + 0.010*"qualidad"

Tópico 3: 0.014*"empresa" + 0.014*"organizacion" + 0.014*"clima" + 0.013*"estilo" + 0.012*"cafe" + 0.011*"alinhamento" + 0.011*"funcionario" + 0.010*"popular" + 0.010*"individuo" + 0.010*"processo"

Tópico 2: 0.048*"bi

### Tópicos mais relevantes por TCC

In [22]:
# Inicializar o data frame auxiliar
data_aux = pd.DataFrame()

# Loop para achar o tópico mais relevantes por documento
for i, resumo in enumerate(lda_model[corpus]):
    resumo = sorted(resumo, key=lambda x: (x[1]), reverse=True)
    for h, (topico, relevancia) in enumerate(resumo):
        if h == 0:
            data_aux = data_aux.append(pd.Series([topico, relevancia]), ignore_index = True)
        else:
            break
data_aux.columns = ['Topico', 'Relevancia']

# Adicionar a URL
data_aux = pd.concat([data_aux, pd.Series(ufrgs['dc.identifier.uri'].values.tolist())], axis=1)

# Data frame com o tópico mais relevante para cada TCC
topicos_relevantes = data_aux
topicos_relevantes.columns = ['Topico', 'Relevancia', 'URL']

In [23]:
topicos_relevantes

,Topico,Relevancia,URL
0,3.0,0.989765,http://hdl.handle.net/10183/151281
1,7.0,0.993585,http://hdl.handle.net/10183/151292
2,7.0,0.843901,http://hdl.handle.net/10183/151294
3,8.0,0.943895,http://hdl.handle.net/10183/151250
4,7.0,0.987803,http://hdl.handle.net/10183/151259
5,9.0,0.992687,http://hdl.handle.net/10183/151256
6,2.0,0.990637,http://hdl.handle.net/10183/170127
7,1.0,0.995855,http://hdl.handle.net/10183/158393
8,6.0,0.994145,http://hdl.handle.net/10183/170115
9,5.0,0.993746,http://hdl.handle.net/10183/170113


### TCC mais relevante por tópico

In [25]:
# Iniciar data frame
tccs_relevantes = pd.DataFrame()

# Agrupar tópicos
topicos_agrupados = topicos_relevantes.groupby('Topico')

# Loop para achar TCC mais relevante por tópico
for i, grupo in topicos_agrupados:
    tccs_relevantes = pd.concat([tccs_relevantes, grupo.sort_values(['Relevancia'], ascending=False).head(1)], axis=0)

# Data frame com TCC mais relevante por tópico   
tccs_relevantes.reset_index(drop=True, inplace=True)
tccs_relevantes.columns = ['Topico', "Relevancia", "URL"]

In [26]:
tccs_relevantes

,Topico,Relevancia,URL
0,0.0,0.993271,http://hdl.handle.net/10183/158436
1,1.0,0.995855,http://hdl.handle.net/10183/158393
2,2.0,0.995860,http://hdl.handle.net/10183/158501
3,3.0,0.996623,http://hdl.handle.net/10183/29792
4,4.0,0.994397,http://hdl.handle.net/10183/158402
5,5.0,0.995749,http://hdl.handle.net/10183/158525
6,6.0,0.995855,http://hdl.handle.net/10183/158586
7,7.0,0.994601,http://hdl.handle.net/10183/158398
8,8.0,0.996213,http://hdl.handle.net/10183/151266
9,9.0,0.996364,http://hdl.handle.net/10183/29844
